# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [4]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [5]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [6]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-Ctkqt7vzMXJzJiQfVrcxQdY0sUhSO',
 'object': 'chat.completion',
 'created': 1767404175,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: Bananas are berries, but strawberries aren’t. Botanically, a berry comes from a single ovary; bananas fit that, while strawberries are made from multiple ovaries of a single flower. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 1080,
  'total_tokens': 1091,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 1024,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [7]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: Bananas are berries, but strawberries aren’t. Botanically, a berry comes from a single ovary; bananas fit that, while strawberries are made from multiple ovaries of a single flower. Want another fun fact?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [8]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: Bananas are technically berries. In botany, a berry is a fruit from a single ovary with seeds embedded in the flesh. Bananas fit that, but strawberries aren’t berries at all—they’re aggregate fruits made from many tiny fruits (achenes) on the outside of a single center.'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [10]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini


In [ ]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [9]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [12]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [10]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [11]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here\'s one:\n\nDid you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," is a type of jellyfish that can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and start its life cycle all over again!'

In [ ]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

In [ ]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

Scrape the website and put it in a variable:

In [16]:
from scraper import fetch_website_contents 
from IPython.display import Markdown, display
website = fetch_website_contents("https://edwarddonner.com")

Option 1: Request a new Summary straight from the message

In [20]:
response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Summarize the following website and translate it to spanish (Latin American): " + website}])

print(response.choices[0].message.content) 

**Summary:**

The website belongs to Edward Donner, an entrepreneur and developer who is passionate about artificial intelligence (AI). As the co-founder and CTO of Nebula.io, he aims to apply AI in a significant way by helping people discover their potential. The company has created a platform that uses proprietary Large Language Models (LLMs) for talent management, which has received praise and press coverage. Edward shares his interests and expertise through various online channels, including writing code, DJing, and discussing AI-related topics on platforms like Hacker News.

**Translation to Spanish (Latin American):**

**Bienvenidos a mi hogar**

Esto es

*Conecta Cuatro*
*Sorpréndete*

Un espacio donde las inteligencias artificiales se enfrentan entre sí en un combate de diplomacia y astucia

**Sobre mí**

Hola, soy Ed. Me gusta escribir código y experimentar con LLMs, espero que tú también te gusten. También disfruto mezclando música electrónica (pero estoy muy fuera de práctic

Option 2: Define System and User Prompts, and create a function to call the prompts, and use markdown for the summary

In [21]:
system_prompt = """
You are a helpful assistant that summarizes and translates websites to spanish (Latin American).
Always respond in markdown. 
"""

user_prompt = """
Summarize the following website: 
"""
def summarize_website(website):
    response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt + website}])
    return response.choices[0].message.content

summary = summarize_website(website)
display(Markdown(summary))




**Resumen del sitio web de Edward Donner**

El sitio web se centra en la aplicación de la Inteligencia Artificial (IA) para ayudar a las personas a descubrir su potencial y perseguir su razón de ser. Ed Donner es el creador y CEO de Nebula.io, una startup que utiliza LLMs (Modelos de Lenguaje Basados en Máquina) para ayudar a los reclutadores a encontrar y gestionar talento.

El sitio ofrece diferentes recursos y contenido relacionado con la IA, como artículos de blog, newsletters y información sobre eventos y conferencias. También está presente inmediatamente en algunos canales sociales: LinkedIn, Twitter y Facebook.

* **Misión:** Aplicar la IA para ayudar a las personas a descubrir su potencial y perseguir su razón de ser.
* **Productos/ Servicios:** Nebula.io, una plataforma que utiliza LLMs para ayudar a los reclutadores a encontrar y gestionar talento.
* **Noticias Recientes:**
 + Fecha de lanzamiento del AI Curriculum (18 de mayo de 2025).
 + Participación en el AI Engineering MLOps Track – Deploy AI to Production el 28 de mayo de 2025.
 + El evento más único de un evento vivo de IA, The Unique Energy of an AI Live Event, el 11 de noviembre de 2025.

Este sitio web destaca la importancia de aplicar la IA para mejorar vidas y alcanzar objetivos.